In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import csv
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM

#Encoder-Decoder Model
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer
from transformers import Trainer
from evaluate import load
#Training

from transformers import TrainingArguments
from transformers import  Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

from datasets import Dataset
import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
bertscore = load("bertscore") 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")

/home2/aparna/miniconda3/envs/diffusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#generate
texts= ["A romantic friendship , passionate friendship , or affectionate friendship is a very close but typically non-sexual relationship between friends , often involving a degree of physical closeness beyond that which is common in the contemporary Western societies .	A romantic friendship , passionate friendship or affectionate friendship is a close but non-sexual relationship between friends that often involves a degree of physical and emotional closeness.","The Alberta Junior Hockey League ( AJHL ) is an Alberta-based Junior A ice hockey league that belongs to the Canadian Junior Hockey League ( CJHL ) .	The Alberta Junior Hockey League ( AJHL ) is an Alberta-based Junior A hockey league that belongs to the Canadian Junior Hockey League ( CJHL ) .","In an electric power system , switchgear is composed of electrical disconnect switches , fuses or circuit breakers used to control , protect and isolate electrical equipment .	A switchgear is a combination of electrical switches , fuses or circuit breakers used to control , protect or isolate electrical equipment."]

for text in texts:
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_output = model.generate(inputs, max_length=512, 
        num_beams=10, top_k=120, top_p=0.95, early_stopping=True, num_return_sequences=10)
    print("====================================")
    for i, beam in enumerate(beam_output):
        print("{}: {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))


/home2/aparna/miniconda3/envs/diffusion/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/diffusion/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `120` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


0: A romantic friendship, passionate friendship, or affectionate friendship is a very close but typically non-sexual relationship between friends, often involving a degree of physical and emotional closeness.
1: A romantic friendship, passionate friendship, or affectionate friendship is a very close but usually non-sexual relationship between friends, often involving a degree of physical and emotional closeness.
2: A romantic friendship, passionate friendship, or affectionate friendship is a very close but typically non-sexual relationship between friends. SEP> It often involves a degree of physical and emotional closeness.
3: A romantic friendship, passionate friendship, or affectionate friendship is a very close but usually non-sexual relationship between friends. SEP> It often involves a degree of physical and emotional closeness.
4: A romantic friendship, passionate friendship, or affectionate friendship is a very close but typically non-sexual relationship between friends, often i

In [3]:
#generate
texts= ["Passionate friendship is a close but non-sexual relationship between friends","Electrical switches , fuses or circuit breakers were used to control , protect or isolate electrical equipment.","A hockey league belongs to the Canadian Junior Hockey League"]

for text in texts:
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_output = model.generate(inputs, max_length=512, 
        num_beams=10, top_k=120, top_p=0.95, early_stopping=True, num_return_sequences=10)
    print("====================================")
    for i, beam in enumerate(beam_output):
        print("{}: {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))


0: Passionate friendship is a close but non-sexual relationship between friends.
1: Passionate friendship is a close relationship between friends.
2: Passionate friendship is a close but non-sexual relationship between friends and family.
3: Passionate friendship is a relationship between friends.
4: Passionate friendship is a close but nonsexual relationship between friends.
5: Passionate friendship is a close and non-sexual relationship between friends.
6: Passionate friendship is a close but non-sexual relationship between friends and friends.
7: Passionate friendship is a close friendship between friends.
8: Passionate friendship is a relationship between friends and family.
9: Passionate friendship is a close relationship between friends and family.
0: Electrical switches, fuses or circuit breakers were used to control, protect or isolate electrical equipment.
1: Electrical switches, fuses and circuit breakers were used to control, protect or isolate electrical equipment.
2: Elect

In [8]:
# %%
import numpy

from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from simpletransformers.t5 import T5Model, T5Args
from transformers import pipeline
#import train split
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import torch
import torch.nn as nn
#from google.transliteration import transliterate_word
#import klib
import os
#bleu score
from torchtext.data.metrics import bleu_score
import csv
colnames = ['source', 'target']
input_file = "train.tsv"
#df = pd.read_csv(input_file, sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)
test = pd.read_csv("valid.tsv", sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)
test = Dataset.from_pandas(test)
# %%
#tokenize
#tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<pad>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['</s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<unk>']})

maxlen = 512
def tokenize_df(df):
    target = tokenizer(df['target'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input = tokenizer(df['source'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input_ids = input['input_ids']
    attention_mask = input['attention_mask']
    target_ids = target['input_ids']
    target_attention_mask = target['attention_mask']
    decoder_input_ids = target_ids.clone()
    #convert to tensors
    input_ids = torch.tensor(input_ids).squeeze()
    attention_mask = torch.tensor(attention_mask).squeeze()
    target_ids = torch.tensor(target_ids).squeeze()
    target_attention_mask = torch.tensor(target_attention_mask).squeeze()
   # decoder_input_ids = torch.tensor(decoder_input_ids)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': target_ids,
        #'decoder_input_ids': decoder_input_ids,
        #'decoder_attention_mask': target_attention_mask
    }




# %%
# train = load_dataset('csv', data_files='train.csv')
# val = load_dataset('csv', data_files='val.csv')
# test = load_dataset('csv', data_files='test.csv')
# train = train.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
# val = val.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
test = test.map(tokenize_df, batched=True, batch_size=128,remove_columns=['source','target'])


# %%
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer


bleu_metric = evaluate.load("sacrebleu")

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring
    # (Note : Please change this code, when you perform on other languages except for Japanese)
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    #blue score
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels]
    result = bleu_metric.compute(predictions=texts_bleu, references=labels_bleu)
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    ), result['score']

# %%
from torch.utils.data import DataLoader
#tokenizer = MT5Tokenizer.from_pretrained("./mt5")
def testing(model):
    metrics =[]
    sample_dataloader = DataLoader(
      test.with_format("torch"),
      collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
      batch_size=5)
    for batch in sample_dataloader:
      with torch.no_grad():
        preds = model.generate(
          batch["input_ids"],
          num_beams=15,
          num_return_sequences=1,
          no_repeat_ngram_size=1,
          remove_invalid_values=True,
          max_length=128,
        )
      labels = batch["labels"]
      metric = metrics_func([preds, labels])
      metrics.append(metric)
    return metrics

def average_metric(metrics):
    rouge = 0
    rouge2 = 0
    rougeL = 0
    rougeLsum = 0
    bleu = 0
    for metric in metrics:
        rouge += metric[0]['rouge1']
        rouge2 += metric[0]['rouge2']
        rougeL += metric[0]['rougeL']
        rougeLsum += metric[0]['rougeLsum']
        bleu += metric[1]
    return rouge/len(metrics),rouge2/len(metrics),rougeL/len(metrics),rougeLsum/len(metrics),bleu/len(metrics)
      

# %%

metrics = testing(model)

# %%
print("t5_small")
scores = average_metric(metrics)
print("rouge:",scores[0])
print("rouge2:",scores[1])
print("rougeL:",scores[2])
print("rougeLsum:",scores[3])
print("bleu:",scores[4])


# %%
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  test.with_format("torch"),
  collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"],
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  inputs = batch["input_ids"]
  break

# Replace -100 (see above)
inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
text_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
#print(bleu_score(list(text_labels.split()),list(text_preds.split())))

# Show result
print("***** Input's Text *****")
print(text_inputs[2])
print("***** summmary (True Value) *****")
print(text_labels[2])
print("***** summary (Generated Text) *****")
print(text_preds[2])

# %%
for i in range(5):
    print("***** Input's Text *****")
    print(text_inputs[i])
    print("***** summarry (True Value) *****")
    print(text_labels[i])
    print("***** summary (Generated Text) *****")
    print(text_preds[i])




Map:   0%|          | 0/20000 [00:00<?, ? examples/s]/tmp/ipykernel_20575/1962435399.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).squeeze()
/tmp/ipykernel_20575/1962435399.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask).squeeze()
/tmp/ipykernel_20575/1962435399.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_ids = torch.tensor(target_ids).squeeze()
/tmp/ipykernel_20575/1962435399.py:50: UserWarning: To copy construct from a tensor, i